In [1]:
#from keras.models import load_model
import cv2
import numpy as np
import matplotlib.image as mpimg  
import os

def find_near_four_pixels(row,col,height,width):
    if row == 0 and col == 0:
        neighbours = None
    elif row == 0:
        neighbours = [(row,col-1)]
    elif col == 0:
        neighbours = [(row-1,col),(row-1,col+1)]
    elif col == width - 1:
        neighbours = [(row-1,col),(row-1,col-1),(row,col-1)]
    else:
        neighbours = [(row-1,col),(row-1,col-1),(row-1,col+1),(row,col-1)]
    return neighbours

def cropimage(file):
    os.system('rm -rf result')
    os.system('mkdir result')
    img = cv2.imread(file)
    result = img.copy()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
    thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)[1]
    height,width = thresh.shape
    linked = {}
    labels = np.zeros((height,width))
    
    row = 0
    next_label = 1
    big_array = []
    while row < height:
        col = 0
        while col < width:
            #?????????
            if thresh[row][col] == 0:
                neighbours = find_near_four_pixels(row,col,height,width)
                #?????????)
                temp = []
                for item in neighbours:
                    if thresh[item[0]][item[1]] == 0:
                        neighbours.remove(item)
                        temp.append(item)
                #print(row,col)
                if temp == []:
                    linked[next_label] = [(row,col)]
                    labels[row][col] = next_label
                    next_label += 1
                else:
                    mini = 100000
                    coord = None
                    for item in temp:
                        if labels[item[0]][item[1]] < mini:
                            mini = labels[item[0]][item[1]]
                            coord = item
                    if coord is not None:
                        labels[row][col] = mini
                    #print(neighbours,row,col,height,width)
                    linked[mini].append((row,col))
                    for item in temp:
                        label = labels[item[0]][item[1]]
                        if label != mini:
                            linked[mini] += linked[label]
                            if linked[mini] in big_array:
                                big_array.remove(linked[mini])
                            if linked[mini] not in big_array:
                                big_array.append(linked[mini])
                            if linked[label] in big_array:
                                big_array.remove(linked[label])
            col += 1
        row += 1
    
    #key = [i for i in linked]
    i = 0
    f = open("output.txt","w")
    for component in big_array:
        max_row = 0
        min_row = height
        max_col = 0
        min_col = width
        #print(i+1)
        for item in component:
            if max_row <= item[0]:
                max_row = item[0]
            if min_row >= item[0]:
                min_row = item[0]
            if max_col <= item[1]:
                max_col = item[1]
            if min_col >= item[1]:
                min_col = item[1]
        temp_width = 1 + max_col-min_col
        temp_height = 1 + max_row-min_row
        outputimage = thresh[min_row-2:max_row+2, min_col-2:max_col+2]
        area = temp_width*temp_height
        if area >= 20:
            outputimage = cv2.resize(outputimage,(32,96),interpolation = cv2.INTER_AREA)
            cv2.imwrite("./result/" + str(i) + ".png", outputimage)
            i += 1
          
    labels[labels == 0] = -1
    labels[labels > -1] = 0
    labels[labels == -1] = 255
    cv2.imwrite('graph.png', labels)
    f.close()
    return labels

cropimage('123.png')

array([[255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       ...,
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.]])